In [19]:
import nltk
nltk.download('punkt')
import numpy as np

[nltk_data] Downloading package punkt to /Users/max/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [20]:
# -*- coding: utf-8 -*-


import csv
import random
from wsgiref import headers

from click import option
from language_model_py import *
from operator import itemgetter
from nltk import word_tokenize 

In [4]:
parentdir = "/Users/max/Desktop/NLP/Resources/sentence-completion"
question_s = "/Users/max/Desktop/NLP/Resources/sentence-completion/testing_data.csv"
answer_s =   "/Users/max/Desktop/NLP/Resources/sentence-completion/test_answer.csv"

In [96]:
class question():
    
    def __init__(self,aline):
        self.fields=aline
    
    def get_field(self,field):
        return self.fields[question.colnames[field]]
    
    def add_answer(self,fields):
        self.answer=fields[1]
   
    def chooseA(self):
        return("a")

    def get_tokens(self):
        return ["__START"]+tokenize(self.fields[question.colnames["question"]])+["__END"]

    def get_left_context(self,window = 1, target = "_____"):
        """Get words preceeding a target word"""
        found=-1
        sent_tokens = self.get_tokens()
        for i,token in enumerate(sent_tokens):
            if token==target:
                found=i
                break 
                
        if found>-1:
            return sent_tokens[i-window:i]
        else:
            return []
    
    def get_right_context(self,window = 1, target = "_____"):
        """Get words preceeding a target word"""
        found=-1
        sent_tokens=self.get_tokens()
        for i,token in enumerate(sent_tokens):
            if token==target:
                found=i
                break  
          
        if found>-1:
            
            return sent_tokens[found+1:found+window+1]
           
        else:
            return []

    def chooserandom(self):
        choices=["a","b","c","d","e"]
        return np.random.choice(choices)

    def predict(self,method="chooseA"):

        choices=["a","b","c","d","e"]
        methoda = {"method" : "unigram", "smoothing":"kneser_ney" }
        methodb = {"method" : "bigram", "smoothing":"kneser_ney" }
        #eventually there will be lots of methods to choose from
        #current_question = question(scc_reader.q)
        #q = question()

        headers = [str(ind) for ind,_ in question.colnames.items()]
        #n = len(self.fields)
        #self.choice_answers = [self.get_field(i) for i in headers[2:]]
        #n = self.get_field('id')
        

        if method=="chooseA":
            return self.chooseA()
        else: 

            left_cont = self.get_left_context()
            right_cont = self.get_right_context()

            if method == "random": 
                choosen_option = random.randint(0,4)
                return choices[choosen_option]

            if method == "unigram":
                probs = [(opt, mylm.get_prob(self.get_field(opt +')'), methodparams = methoda)) for opt in choices]     
                return sorted(probs, key = operator.itemgetter(1), reverse = True)[0][0]
            if method == "bigram_l":
                probs = [(opt, mylm.get_prob(self.get_field(opt +')'),context = left_cont[0], methodparams = methodb))  for opt in choices]
                return sorted(probs, key = operator.itemgetter(1), reverse = True)[0][0]
            if method == "bigram_r":
                probs = [(opt, mylm.get_prob(right_cont[0] , context = self.get_field(opt +')'), methodparams = methodb))  for opt in choices]
                return sorted(probs, key = operator.itemgetter(1), reverse = True)[0][0]
            if method == "bigram":
                probs = [(opt,  mylm.get_prob(self.get_field(opt +')'), context = left_cont[0], methodparams = methodb) * mylm.get_prob(right_cont[0], context = self.get_field(opt +')'), methodparams = methoda))  for opt in choices]
                return sorted(probs, key = operator.itemgetter(1), reverse = True)[0][0]

# get scc_mod working then Add triagrams in language model and


    def predict_and_score(self,method="chooseA"):
        
        #compare prediction according to method with the correct answer
        #return 1 or 0 accordingly
        prediction=self.predict(method=method)
        if prediction ==self.answer:
            return 1
        else:
            return 0

class scc_reader():
    
    def __init__(self,qs=question_s,ans=answer_s):
        self.qs=qs
        self.ans=ans
        self.read_files()
        
    def read_files(self):
        
        #read in the question file
        with open(self.qs) as instream:
            csvreader=csv.reader(instream)
            self.qlines=list(csvreader)
        
        #store the column names as a reverse index so they can be used to reference parts of the question
        question.colnames={item:i for i,item in enumerate(self.qlines[0])}
        #id = 0, question = 1, a = 2, b =3, c =4, d =5, e=6
        
        #create a question instance for each line of the file (other than heading line)
        scc_reader.questions=[question(qline) for qline in self.qlines[1:]]
        
        #read in the answer file
        with open(self.ans) as instream:
            csvreader=csv.reader(instream)
            alines=list(csvreader)
            
        #add answers to questions so predictions can be checked    
        for q,aline in zip(self.questions,alines[1:]):
            q.add_answer(aline)
        
    def get_field(self,field):
        return [q.get_field(field) for q in scc_reader.questions] 
    
    def predict(self,method="chooseA"):
        return [q.predict(method=method) for q in scc_reader.questions]
    
    def predict_and_score(self,method="chooseA"):
        scores=[q.predict_and_score(method=method) for q in scc_reader.questions]
        return sum(scores)/len(scores)

In [98]:
SCC = scc_reader()

In [99]:
SCC.predict_and_score(method="bigram")

0.17307692307692307

In [87]:
one_q = question(SCC.qlines[1])
one_q.get_tokens()
one_q.get_right_context()

['alone']

In [74]:
SCC.predict_and_score(method = "bigram_r")

IndexError: list index out of range